<a href="https://colab.research.google.com/github/iskra3138/colab_seminar/blob/master/autogluon_Image_Classification_How_to_Use_Your_Own_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## For P100
!nvidia-smi

# AutoGluon 설치

In [0]:
!pip install --upgrade mxnet-cu100
!pip install autogluon

RESTART RUNTIME 버튼을 누르지 말고 아래 에러 해결을 위한 code cell 하나 더 실행하고 RESTART

- task.fit 실행시 'ValueError: max() arg is an empty sequence' 발생 시 아래 code cell 실행
  - 출처 :<https://github.com/awslabs/autogluon/issues/163>

In [0]:
!pip uninstall -y distributed
!pip install distributed
!pip install -U ipykernel

- output 제일 아래 RESTART RUNTIME 버튼을 눌러서 런타임 다시 시작하고 아래 부터 실행

# Image Classification - How to Use Your Own Datasets

[출처] <https://autogluon.mxnet.io/tutorials/image_classification/kaggle.html>

This tutorial demonstrates how to use AutoGluon with your own custom datasets. As an example, we use a dataset from Kaggle to show the required steps to format image data properly for AutoGluon.

### Step 1: Organizing the dataset into proper directories

After completing this step, you will have the following directory structure on your machine:

In [0]:
'''
data/
├── class1/
├── class2/
├── class3/
├── ...
'''

Here data is a folder containing the raw images categorized into classes. For example, subfolder class1 contains all images that belong to the first class, class2 contains all images belonging to the second class, etc. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

Under each class, the following image formats are supported when training your model:
- JPG
- JPEG
- PNG

In the same dataset, all the images should be in the same format. Note that in image classification, we do not require that all images have the same resolution.

You will need to organize your dataset into the above directory structure before using AutoGluon. Below, we demonstrate how to construct this organization for a Kaggle dataset.

#### Kaggle과 Google colab 연결하기

[출처] <https://towardsdatascience.com/setting-up-kaggle-in-google-colab-ebb281b61463>

First, grab your token from Kaggle.

Go to your account page (the drop-down menu in the top right corner of the screen will take you there).
![대체 텍스트](https://miro.medium.com/max/2542/1*f94G6dXm-8cRxX5FFTCqyg.png)

Then scroll down to API and hit “Create New API Token.”
![대체 텍스트](https://miro.medium.com/max/2050/1*rhsGKThdgxwUoskaACSlwA.png)

That’s going to download a file called kaggle.json.
Open the file and you’ll see something that looks a lot like this:
```
{“username”:”YOUR-USER-NAME”,”key”:”SOME-VERY-LONG-STRING”}
```



확인한 자기 token 정보를 이용하여 /root/.kaggle/ directory에 kaggle.json 파일 생성

In [0]:
import os
import json

if not os.path.exists('/root/.kaggle/'):
  os.makedirs('/root/.kaggle')
  
#token = {“username”:”YOUR-USER-NAME”,”key”:”SOME-VERY-LONG-STRING”}
token =  {"username":"iskra3138","key":"875d24d34640942187d5cdc8e8d6d2cf"}#@param {type:"raw"}

assert token, 'For this part, you need an information in the kaggle.json like {“username”:”YOUR-USER-NAME”,”key”:”SOME-VERY-LONG-STRING”}'

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

#### Example: Kaggle dataset

Kaggle is a popular machine learning competition platform and contains lots of datasets for different machine learning tasks including image classification. If you don’t have Kaggle account, please register one at [Kaggle](https://www.kaggle.com/). Then, please follow the [Kaggle installation](https://github.com/Kaggle/kaggle-api/) to obtain access to Kaggle’s data downloading API.

To find image classification datasets in Kaggle, let’s go to [Kaggle](https://www.kaggle.com/) and search using keyword image classification either under Datasets or Competitions.

For example, we find the Shopee-IET Machine Learning Competition under the InClass tab in Competitions.

We then navigate to [Data](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data) to download the dataset using the Kaggle API. Please make sure to click the button of “I Understand and Accept” before downloading the data.

An example shell script to download the dataset to ./data/shopeeiet/ can be found here: [download_shopeeiet.sh](https://github.com/zhanghang1989/AutoGluonWebdata/blob/master/docs/tutorial/download_shopeeiet.sh?raw=True).

- 아래 code cell을 실행시키면 데이터 다운로드가 안됨.
- kaggle 홈페이지에서 shopee-iet-machine-learning-competition 검색한 후 아래 설명대로 해서 해결
  - kaggle 시스템의 특성 상 실제 경쟁에 참여하므로 rule에 대해 인정하는 과정이 없으면 아예 API가 막히는거 같다. 
  - 그래서 해당 competition 페이지를 방문해 보면 Rules 메뉴 탭을 선택하면 다음과 같이 룰을 이해하고 받아들이는 버튼이 있다. 
  - 이 버튼을 누르면서 인정 과정을 거친다.
  - ![대체 텍스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2F0NOHH%2Fbtqv7YZN5gx%2FfJjR0E5CEzfl1Xqcwr6TK1%2Fimg.png)
  - [출처] https://curioso365.tistory.com/98

After downloading this script to your machine, run it with:
- shell script가 작동하지 않아서 따로 풀어서 코드 정리했습니다.

In [0]:
import autogluon as ag

## shell script가 작동하지 않아서 아래 코드로 변경해서 데이터 다운로드
#ag.download('https://raw.githubusercontent.com/zhanghang1989/AutoGluonWebdata/master/docs/tutorial/download_shopeeiet.sh')
#!sh download_shopeeiet.sh

In [0]:
mkdir -p ./data/shopeeiet/train

In [0]:
cd data/shopeeiet/

In [0]:
!kaggle competitions download -c shopee-iet-machine-learning-competition -p ./

In [0]:
!mv Training%20Images.zip train && cd train && unzip Training%20Images.zip

In [0]:
cd /content/data/shopeeiet

In [0]:
!unzip Test%20Images.zip

In [0]:
!mv Test test

In [0]:
!rm *.zip *.csv train/*.zip

In [0]:
cd /content/

Tranin Data 내에 인식되지 않는 이미지들이 있어서 학습이 오류가 나기 때문에 제거
- 예. '/content/data/shopeeiet/train/BabyBibs/BabyBibs_1.jpg'

In [0]:
import os
import mxnet as mx
import shutil

unrecog_dir='/content/data/shopeeiet/unrecognized'
if not os.path.exists(unrecog_dir):
  os.makedirs(unrecog_dir)
for (path, dir, files) in os.walk("/content/data/shopeeiet/train/"):
  for filename in files:
    img_path = os.path.join(path,filename)
    try:
      mx.image.imread(img_path)
    except:
      print (img_path)
      shutil.move(img_path, unrecog_dir)
    '''ext = os.path.splitext(filename)[-1]
      if ext == '.':
        print("%s/%s" % (path, filename))'''

Now we have the desired directory structure under ./data/shopeeiet/train/, which in this case looks as follows:
```
shopeeiet/train
    ├── BabyBibs
    ├── BabyHat
    ├── BabyPants
    ├── ...
shopeeiet/test
    ├── ...
```

Here are some example images from this data:

![대체 텍스트](https://autogluon.mxnet.io/_images/shopeeiet_example.png)

### Step 2: Split data into training/validation sets

A fundamental step in machine learning is to split the data into disjoint sets used for different purposes.

**Training Set**: The majority of your data should be in the training set. This is the data use to train your model: data is used to learn the parameters of the model, namely the weights of the neural network classifier.

**Validation Set**: A separate validation set (sometimes also called the dev set) is also used during AutoGluon’s training process. While neural network weights are updated based on the training data, each neural network requires the user to specify many hyperparameters (e.g., learning rates, etc.). The choice of hyperparameters greatly impacts the training process and resulting model. AutoGluon automatically tries many different values of these hyperparameters and evaluates each hyperparameter setting by measuring the performance of the resulting network on the validation set.

**Test Set**: A separate set of images, possibly without available labels. These data are never used during any part of the model construction or learning process. If unlabeled, these may correspond to images whose labels we would like to predict. If labeled, these images may correspond to images we reserve for estimating the performance of our final model.

#### Automatic training/validation split

AutoGluon automatically does Training/Validation split:

In [0]:
from autogluon import ImageClassification as task
dataset = task.Dataset('/content/data/shopeeiet/train')

AutoGluon automatically infers how many classes there are based on the directory structure. By default, AutoGluon automatically constructs the training/validation set split:

- Training Set: 80% of images.
- Validation Set: 20% of images.

where the images that fall into the validation set are randomly chosen from the training data based on the class.

### Step 3: Use AutoGluon fit to generate a classification model

Now that we have a Dataset object, we can use AutoGluon’s default configuration to obtain an image classification model using the `fit </api/autogluon.task.html#autogluon.task.ImageClassification.fit>`__ function.

Due to the large size of the Kaggle dataset, calling fit without specifying a time limit may result in long waiting times. Run the following commands to run fit using a time limit:

P100에서 약 7-8시간 소요??

In [0]:
#@title
time_limits = 10 * 60 # 10mins
classifier = task.fit(dataset, time_limits=time_limits,
                      ngpus_per_trial=1)

The top-1 accuracy of the best model on the validation set is:

In [0]:
print('Top-1 val acc: %.3f' % classifier.results['best_reward'])

#### Using AutoGluon to Generate Predictions on Test Images

We can ask our final model to generate predictions on the provided test images. We first load the test data as a Dataset object and then call predict:

In [0]:
test_dataset = task.Dataset('./data/shopeeiet/test')
inds, probs, probs_all = classifier.predict(test_dataset)

inds above contains the indices of the predicted class for each test image. probs contains the confidence in these class predictions. probs_all contains the confidence in the whole of classes predictions.

Here are the results of AutoGluon’s default fit and predict under different time_limits when executed on a p3.16xlarge EC2 instance:

- The validation top-1 accuracy within 5h is 0.842, and ranks 14th place in Kaggle competition.
- The validation top-1 accuracy within 24h is 0.846, and ranks 12th place in Kaggle competition.
- The validation top-1 accuracy within 72h is 0.852, and ranks 9th place in Kaggle competition.

### Step 4: Submit test predictions to Kaggle

If you wish to upload the model’s predictions to Kaggle, here is how to convert them into a format suitable for a submission into the Kaggle competition:

In [0]:
import autogluon as ag
ag.utils.generate_csv(inds, './data/shopeeiet/submission.csv')

This produces a submission file located at: ./data/shopeeiet/submission.csv.

To see an example submission, check out sample submission.csv at this link: [Data](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data).

To make your own submission, click [Submission](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/submit) and then follow the steps in the submission page (upload submission file, describe the submission, and click the Make Submission button). Let’s see how your model fares in this competition!